In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_excel('/kaggle/input/dataset/Dummy Dataset for Challenge2.xlsx')

In [ ]:
df.isna()

In [ ]:
df.tail()

In [ ]:
df.isna().sum()

In [ ]:
# Clean data

categorical_columns = ['VARIANT', 'TYPE', 'FRAGRANCE', 'FLAVOUR', 'ADD-ON', 'TAGLINE']
for column in categorical_columns:
    df.fillna({column: 'Unknown' if column != 'ADD-ON' else 'None'}, inplace=True)

In [ ]:
df

In [ ]:
df.duplicated(subset=['ITEM_NAME', 'PACK SIZE'])
df_clean = df.drop_duplicates(subset=['ITEM_NAME', 'PACK SIZE'], keep="first")
df_clean

In [ ]:
new_df = df_clean.loc[:, ['CATEGORY', 'ITEM_NAME', 'SEGMENT', 'PACK SIZE', 'PACKAGE TYPE', 'BRAND']]

In [ ]:
new_df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from collections import Counter

In [ ]:
# Text processing and feature extraction

vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(new_df['ITEM_NAME'])

In [ ]:
X

In [ ]:
similarity_matrix = cosine_similarity(X)

In [ ]:
# Clustering

n_clusters = min(8, new_df.shape[0])
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(similarity_matrix)
new_df['CLUSTER'] = kmeans.labels_

In [ ]:
# Name Recommendation

def get_recommendation(cluster):
    names = new_df[new_df['CLUSTER'] == cluster]['ITEM_NAME']
    most_common_name = Counter(names).most_common(1)[0][0]
    return most_common_name

# Apply the recommendation to each cluster
new_df['RECOMMENDED_NAME'] = new_df['CLUSTER'].apply(get_recommendation)

In [ ]:
new_df

In [ ]:
# Display

print(new_df[['ITEM_NAME', 'RECOMMENDED_NAME']])